# Setup Code (Run first)


In [234]:
%%capture
# Capture the output but do not do anything with it
# Just to avoid cluttering the output

%pip install python-terrier==0.9.2
%pip install semanticscholar
%pip install gensim
%pip install seaborn
%pip install KrovetzStemmer
%pip install nltk
%pip install langdetect
%pip install Unidecode
%pip install pandas
%pip install pymed

# Load all packages and initialize pyTerrier
from langdetect import detect
from unidecode import unidecode
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from krovetzstemmer import Stemmer
from semanticscholar import SemanticScholar
from gensim.models import Word2Vec
from itertools import islice, chain
from tqdm import tqdm
from pyterrier.measures import *
from pymed import PubMed
import requests
import os
import pyterrier as pt
import pandas as pd
import numpy as np
import nltk
import time
import string

In [ ]:
# If not done already download the required data
nltk.download()

# Vorgehen

## Allgemein

TREC-Covid-Queries verwenden und erweitern um weitere Anfrageterme, um die Retrieval-Performance zu verbessern (Query Expansion).

Für die QE müssen Termkandidaten ("set of C" c_1, c_2, c_3, ...) bestimmt werden, die anschließend gerankt werden.
D.h. ihr sendet zunächst die Standard-Query ab und später nochmal für die finale Evaluierung, die Query mit Termerweiterungen.

## QE

Wenn wenig Zeit:
Nur die globale Variante evaluieren, wobei ja der "Claim for Fame" im Paper ist, dass die lokale Methode bessere Ergebnisse liefert.

### LOKALE QE

Word2Vec-Embeddings auf TREC-Covid trainieren (https://radimrehurek.com/gensim/)


### GLOBALE QE

Standardvariante wie z.B. auf Basis von Wikipedia.


### Just for testing: Google News word2vec

In [ ]:
# Load the word2vec model
from gensim.models import KeyedVectors
from gensim import models
global_model = models.KeyedVectors.load_word2vec_format("external_models/GoogleNews-vectors-negative300.bin", binary=True)
#global_model = Word2Vec.load("external_models/model.bin")



[('flu_virus', 0.779543936252594),
 ('viruses', 0.768750011920929),
 ('H#N#_virus', 0.7382203340530396)]

In [ ]:
w2v_model.most_similar('coughing', topn=3)

[('cough', 0.6782324910163879),
 ('coughs', 0.6672220826148987),
 ('sneezing', 0.6352056860923767)]

### Global QE GLOVE Wiki Gigaword
Wikipedia 2014 + Gigaword 5 

# 1. Standard-Query an den Index für erstes Ranking

Aus dem Title des Topics, quasi wie in der Standard-Pipeline in Pyterrier.
Paper Inverse Document Frequency model (InL2)
Zuerst BM25.


# 2. Erstellung der Termkandidaten

## 2.1 Top-3 Dokumente

Alle Terme (ausschließlich der Stoppworte).


## 2.2 Terme der Referenzen

Alle Terme aus den Referenzen der 3 Dokumente (Terme der Titel).


## 2.3 Erweiterung mit Co-Autoren

Über die Koautoren können noch weitere Dokumente bzw. die entsprechenden Terme hinzugefügt werden. Aus Zeitgründen könnte man vielleicht auch diesen Schritt weglassen, wenn es zu viel Aufwand ist über Koautoren weitere Dokumente zu finden. Bei der Implementierung scheint ihr ja aber schon recht weit zu sein. Die "relevanten Paper der Autoren" sind einfach die Top-k Dokumente oder möglicherweise alle zusätzlichen Papers, die über die Koautorenschaft gefunden werden, gemeint. Daher ergibt sich auf der Name PSEUDO-Relevanz-Feedback, da einfach angenommen wird, dass die Top-Treffer alle relevant sind.

Referenzen = Quellen (Alle im Paper direkt zitierten Quellen)

Paper der Co-Autoren = Alle Autoren des Papers und deren Paper



# 2.4 Ranking aller Terme



Paper: Bo1
Top-k Terme auswählen (k selbst wählen).
Ergebnis: Potentielle Kandidaten für eine Termwerweiterung

Bezug zur Mail:

Bei 2.1.4
    Wir nehmen alle Terme (topK abstracts, topK references titles, topK authors relevant papers abstracts) und ranken mit bo1.
    Nehmen topK (3)

Bei 2.2.1
    Wir nehmen alle Terme (s.o.)
    Trainieren Word2Vec
    bo1 ranken
    Nehmen topK (3)

=> Kombinieren und topK nehmen und als Queries absenden.


# 2.5 Word2Vec-Modell mit Terms nachtrainieren

Über die Kosinusähnlichkeit der Embeddings der ursprünglichen Anfrageterme bestimmt ihr nun weitere Terme aus dem Word2Vec-Modell. Diese Termkandidaten werden dann wie in 2.1.4 über Bo1 o.ä. gerankt.


In [257]:
# Ranking
bm25 = pt.BatchRetrieve(trec_covid_index, wmodel='BM25')
bo1 = pt.rewrite.Bo1QueryExpansion(trec_covid_index)
pipelineQE = bm25 >> bo1 >> bm25

res = pipelineQE.transform(all_terms_expanded_df)
all_terms_expanded_ranked_results = res.sort_values(by="score", ascending=False)

In [259]:
all_terms_expanded_ranked_results

,qid,docid,docno,rank,score,query_0,query
1046469,2124,109806,k1sh5aqd,1,89.591816,han,applypipeline:off han^1.624118669 sufrido^0.96...
1046470,2124,109807,0odga0w9,2,89.591816,han,applypipeline:off han^1.624118669 sufrido^0.96...
1046468,2124,109805,ndco6jv2,0,89.591816,han,applypipeline:off han^1.624118669 sufrido^0.96...
4803964,6161,114562,vcsu27al,0,82.417343,form,applypipeline:off form^1.737729974 watersh^0.8...
3209808,4456,114562,vcsu27al,0,82.417343,form,applypipeline:off form^1.737729974 watersh^0.8...
...,...,...,...,...,...,...,...
4838390,6199,2272,e7xwb03g,426,0.000000,send,applypipeline:off send^1.637534024 so^0.000000...
4838389,6199,2271,x4z0uj0g,425,0.000000,send,applypipeline:off send^1.637534024 so^0.000000...
4838388,6199,2265,38rmx65j,424,0.000000,send,applypipeline:off send^1.637534024 so^0.000000...
4838387,6199,2262,a4bgut8h,423,0.000000,send,applypipeline:off send^1.637534024 so^0.000000...


# 2.6 Zwei Sets zusammenfügen und Top-k für QE der ursprünglichen Query nutzen

Diese jeweils für die 50 Topics absenden.

Ergebnis: Die finalen Rankings, die dann ausgewertet werden können.
